In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
bdi2_df = pd.read_excel(r"C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Projects\teis-mikeriello\data\NSS data_BDI2 scores_20200101-20230131_20241106.xlsx")

bdi2_df['testDate'] = pd.to_datetime(bdi2_df['testDate'])

In [5]:
bdi3_df = pd.read_excel(r"C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Projects\teis-mikeriello\data\NSS data_BDI3 Scores 20230101 20240630_Compiled_20241107.xlsx")

bdi3_df['Adaptive-Self Care Date of Testing'] = pd.to_datetime(bdi3_df['Adaptive-Self Care Date of Testing'])

In [7]:
eco_df = pd.read_excel(r"C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Projects\teis-mikeriello\data\NSS_ECO Data Ind 3 FFY 2023-24 20241211.xlsx",sheet_name='ECO with Exit23-24 ')

In [9]:
entry_exit = eco_df[(eco_df['OC1 Data Check']==1)][['CHILD_ID','BDI 3\nECO_Entry_DATE','BDI 3\nECO_Exit_DATE','BDI2 Entry Date']]

In [11]:
entry_exit.rename(columns={'CHILD_ID':'TEIDS Child ID'}, inplace=True)

In [13]:
entry_exit = entry_exit[(entry_exit['BDI 3\nECO_Exit_DATE'].notnull())&((entry_exit['BDI 3\nECO_Entry_DATE'].notnull())|(entry_exit['BDI2 Entry Date'].notnull()))]

In [15]:
bdi3_exit = entry_exit.merge(bdi3_df,how='inner',left_on=['TEIDS Child ID','BDI 3\nECO_Exit_DATE'],right_on=['TEIDS Child ID','Adaptive-Self Care Date of Testing'])

In [17]:
bdi3_entry = entry_exit.merge(bdi3_df,how='inner',left_on=['TEIDS Child ID','BDI 3\nECO_Entry_DATE'],right_on=['TEIDS Child ID','Adaptive-Self Care Date of Testing'])

In [19]:
bdi2_entry_subdomains = entry_exit.merge(bdi2_df,how='inner',left_on=['TEIDS Child ID','BDI2 Entry Date'],right_on=['Student TEIDS ID','testDate'])

In [21]:
bdi2_entry_subdomains = bdi2_entry_subdomains[~bdi2_entry_subdomains['testDate'].isnull()]

In [23]:
bdi2_entry = bdi2_df[bdi2_df['studentAssessmentID'].isin(bdi2_entry_subdomains['studentAssessmentID'])]

In [25]:
bdi2_entry['Student TEIDS ID'] = bdi2_entry['Student TEIDS ID'].astype('int64')

C:\Users\mrmik\AppData\Local\Temp\ipykernel_178664\2447718271.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bdi2_entry['Student TEIDS ID'] = bdi2_entry['Student TEIDS ID'].astype('int64')


In [27]:
bdi3_entry = bdi3_entry[~bdi3_entry['Adaptive-Self Care Date of Testing'].isnull()]

In [29]:
bdi2_entry = bdi2_entry[['Student TEIDS ID','domainText','rawScore','DQScore','ZScore']]

In [31]:
bdi3_entry = bdi3_entry[['TEIDS Child ID','Adaptive Developmental Quotient','Social-Emotional Developmental Quotient','Communication Developmental Quotient','Motor Developmental Quotient','Cognitive Developmental Quotient','Adaptive-Self Care RS','Adaptive-Self Care Z-Score','Adaptive-Personal Responsibility RS','Adaptive-Personal Responsibility Z-Score','Social Emotional-Adult Interaction RS','Social Emotional-Adult Interaction Z-Score','Social Emotional-Peer Interaction RS','Social Emotional-Peer Interaction Z-Score','Social Emotional-Self Concept / Social Role RS','Social Emotional-Self Concept / Social Role Z-Score','Communication-Receptive Communication RS','Communication-Receptive Communication Z-Score','Communication-Expressive Communication RS','Communication-Expressive Communication Z-Score','Motor-Gross Motor RS','Motor-Gross Motor Z-Score','Motor-Fine Motor RS','Motor-Fine Motor Z-Score','Motor-Perceptual Motor RS','Motor-Perceptual Motor Z-Score','Cognitive-Attention and Memory RS','Cognitive-Attention and Memory Z-Score','Cognitive-Reasoning / Academic Skills RS','Cognitive-Reasoning / Academic Skills Z-Score','Cognitive-Perception and Concepts RS','Cognitive-Perception and Concepts Z-Score']]

In [33]:
bdi3_exit = bdi3_exit[['TEIDS Child ID','Adaptive Developmental Quotient','Social-Emotional Developmental Quotient','Communication Developmental Quotient','Motor Developmental Quotient','Cognitive Developmental Quotient','Adaptive-Self Care RS','Adaptive-Self Care Z-Score','Adaptive-Personal Responsibility RS','Adaptive-Personal Responsibility Z-Score','Social Emotional-Adult Interaction RS','Social Emotional-Adult Interaction Z-Score','Social Emotional-Peer Interaction RS','Social Emotional-Peer Interaction Z-Score','Social Emotional-Self Concept / Social Role RS','Social Emotional-Self Concept / Social Role Z-Score','Communication-Receptive Communication RS','Communication-Receptive Communication Z-Score','Communication-Expressive Communication RS','Communication-Expressive Communication Z-Score','Motor-Gross Motor RS','Motor-Gross Motor Z-Score','Motor-Fine Motor RS','Motor-Fine Motor Z-Score','Motor-Perceptual Motor RS','Motor-Perceptual Motor Z-Score','Cognitive-Attention and Memory RS','Cognitive-Attention and Memory Z-Score','Cognitive-Reasoning / Academic Skills RS','Cognitive-Reasoning / Academic Skills Z-Score','Cognitive-Perception and Concepts RS','Cognitive-Perception and Concepts Z-Score']]

In [35]:
# Dibran wrote this, don't praise me. -- Ryan
def transform_data(file)-> pd.DataFrame:
    df = file #This was originally where you would read in the csv for bdi-2, I changed it to run the already filtered dataframe -- Ryan
        #drop null student ID if you have any
    df = df[~df['Student TEIDS ID'].isnull()]
    # there are ids that have mulitple duplicate IDs example:526857
    #the following code will help deal with duplciate domain - allowing us to create more that 1 row if it has dup domain text
    df = df.sort_values(['Student TEIDS ID','domainText']).reset_index(drop=True)
    #sequential number to each row within a group
    df['row_number'] = df.groupby(['Student TEIDS ID', 'domainText']).cumcount()
    #pivot dataframe
    df_pivot = df.pivot_table(index=['Student TEIDS ID', 'row_number']
              , columns='domainText'
              , values=['rawScore','DQScore','ZScore']
              , aggfunc='first')
    #flatten multi-index columns
    df_pivot.columns = [f"{col[1]}-{col[0]}" for col in df_pivot.columns]
    #reset index
    df_pivot = df_pivot.reset_index()
    #drop row number
    df_pivot = df_pivot.drop(columns=['row_number'])
    return df_pivot

In [37]:
bdi2_df_test = transform_data(bdi2_entry)

In [39]:
bdi2_df_test

,Student TEIDS ID,Adaptive-DQScore,Cognitive-DQScore,Communication-DQScore,Motor-DQScore,Personal-Social-DQScore,Adaptive-ZScore,Adult Interaction-ZScore,Attention and Memory-ZScore,Cognitive-ZScore,...,Motor-rawScore,Peer Interaction-rawScore,Perception and Concepts-rawScore,Perceptual Motor-rawScore,Personal Responsibility-rawScore,Personal-Social-rawScore,Reasoning and Academic Skills-rawScore,Receptive Communication-rawScore,Self-Care-rawScore,Self-Concept and Social Role-rawScore
0,474848,75.0,63.0,55.0,90.0,70.0,-1.67,-2.00,-1.67,-2.47,...,77.0,NaN,8.0,NaN,NaN,33.0,NaN,11.0,23.0,10.0
1,474967,80.0,80.0,73.0,70.0,78.0,-1.33,-0.33,-2.00,-1.33,...,2.0,NaN,4.0,NaN,NaN,5.0,NaN,2.0,2.0,0.0
2,475907,90.0,73.0,78.0,85.0,85.0,-0.67,-0.67,-2.67,-1.80,...,5.0,NaN,4.0,NaN,NaN,6.0,NaN,6.0,3.0,2.0
3,476193,100.0,83.0,75.0,93.0,88.0,0.00,0.00,-1.33,-1.13,...,9.0,NaN,4.0,NaN,NaN,9.0,NaN,7.0,4.0,2.0
4,478004,85.0,73.0,75.0,80.0,80.0,-1.00,-1.33,-3.00,-1.80,...,7.0,NaN,6.0,NaN,NaN,7.0,NaN,4.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2660,545411,80.0,90.0,62.0,78.0,88.0,-1.33,0.00,-0.33,-0.67,...,67.0,NaN,12.0,NaN,NaN,49.0,NaN,15.0,25.0,13.0
2661,545428,105.0,82.0,70.0,102.0,83.0,0.33,-1.00,-1.33,-1.20,...,105.0,9.0,14.0,16.0,11.0,64.0,8.0,24.0,34.0,20.0
2662,545473,83.0,82.0,68.0,111.0,92.0,-1.13,0.00,-1.00,-1.20,...,109.0,12.0,13.0,18.0,5.0,77.0,7.0,19.0,28.0,24.0
2663,545599,75.0,90.0,86.0,75.0,95.0,-1.67,0.00,-0.33,-0.67,...,35.0,NaN,8.0,NaN,NaN,31.0,NaN,12.0,9.0,10.0


In [41]:
test_ids = bdi3_exit[(bdi3_exit['TEIDS Child ID'].isin(bdi3_entry['TEIDS Child ID']))|(bdi3_exit['TEIDS Child ID'].isin(bdi2_entry['Student TEIDS ID']))]['TEIDS Child ID']

In [43]:
test_ids

0       474848
1       474967
2       475907
3       476193
4       478004
         ...  
3908    564474
3909    564596
3910    565093
3911    565305
3912    565335
Name: TEIDS Child ID, Length: 3842, dtype: int64

In [45]:
#(bdi3_exit[(bdi3_exit['TEIDS Child ID'].isin(bdi3_entry['TEIDS Child ID']))|(bdi3_exit['TEIDS Child ID'].isin(bdi2_entry['Student TEIDS ID']))]['TEIDS Child ID']).to_csv('TEIDS Child IDs',index=False)

In [47]:
bdi2_flag = pd.DataFrame({'TEIDS Child ID':bdi2_entry['Student TEIDS ID'].unique()})
bdi2_flag['bdi_flag'] = 2

In [49]:
bdi3_flag = pd.DataFrame(bdi3_entry['TEIDS Child ID'])
bdi3_flag['bdi_flag'] = 3

In [51]:
entry_flag = bdi2_flag.merge(bdi3_flag,how='outer',left_on=['TEIDS Child ID','bdi_flag'],right_on=['TEIDS Child ID','bdi_flag'])

In [53]:
entry_flag

,TEIDS Child ID,bdi_flag
0,474848,2
1,474967,2
2,475907,2
3,476193,2
4,478004,2
...,...,...
4051,567392,3
4052,567510,3
4053,567756,3
4054,568323,3


In [55]:
exit_flagged = entry_flag.merge(bdi3_exit,how='inner',on='TEIDS Child ID')

In [57]:
bdi2_entry_exit = exit_flagged[exit_flagged['bdi_flag']==2]

In [59]:
bdi3_entry_exit = exit_flagged[exit_flagged['bdi_flag']==3]

In [61]:
#domains = pd.DataFrame({'Adaptive':['Self-Care','Personal Responsibility'],'Social-Emotional':['Adult Interaction','Peer Interaction','Self-Concept and Social Role'],'Communication':['Receptive Communication','Expressive Communication'],'Motor':['Gross Motor','Fine Motor','Perceptual Motor'],'Cognitive':['Attention and Memory','Reasoning and Academic Skills','Perception and Concepts']})

In [63]:
#OC1 = ['*Social-Emotional*','*Personal-Social*','*Adult Interaction*','*Peer Interaction*','*Self-Concept and Social Role*','*Self Concept / Social Role*']

In [65]:
bdi3_entry

,TEIDS Child ID,Adaptive Developmental Quotient,Social-Emotional Developmental Quotient,Communication Developmental Quotient,Motor Developmental Quotient,Cognitive Developmental Quotient,Adaptive-Self Care RS,Adaptive-Self Care Z-Score,Adaptive-Personal Responsibility RS,Adaptive-Personal Responsibility Z-Score,...,Motor-Fine Motor RS,Motor-Fine Motor Z-Score,Motor-Perceptual Motor RS,Motor-Perceptual Motor Z-Score,Cognitive-Attention and Memory RS,Cognitive-Attention and Memory Z-Score,Cognitive-Reasoning / Academic Skills RS,Cognitive-Reasoning / Academic Skills Z-Score,Cognitive-Perception and Concepts RS,Cognitive-Perception and Concepts Z-Score
0,486692,67.0,106.0,79.0,94.0,92.0,27.0,1.67,2.0,2.00,...,34.0,0.67,12.0,0.33,28.0,0.67,14.0,0.33,13.0,1.00
1,490073,67.0,88.0,64.0,70.0,64.0,19.0,2.00,2.0,1.67,...,28.0,0.67,4.0,3.00,21.0,1.67,3.0,2.67,10.0,1.67
2,490134,70.0,88.0,49.0,87.0,84.0,24.0,1.67,3.0,1.67,...,30.0,0.33,12.0,0.33,27.0,0.67,9.0,0.67,12.0,1.33
3,490333,79.0,94.0,70.0,94.0,84.0,31.0,1.00,5.0,1.33,...,26.0,1.67,19.0,1.67,26.0,1.00,10.0,0.67,14.0,1.00
4,490333,97.0,94.0,76.0,98.0,88.0,34.0,0.67,13.0,0.33,...,31.0,0.33,16.0,0.67,27.0,0.67,9.0,0.67,15.0,0.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1388,567392,91.0,98.0,52.0,85.0,76.0,29.0,1.00,10.0,0.00,...,28.0,1.00,12.0,0.33,24.0,1.00,8.0,1.00,10.0,2.00
1389,567510,58.0,86.0,85.0,91.0,100.0,24.0,1.67,0.0,3.00,...,34.0,1.00,12.0,0.33,23.0,1.33,12.0,0.33,23.0,1.00
1390,567756,61.0,66.0,46.0,66.0,72.0,18.0,2.33,2.0,2.00,...,25.0,1.67,6.0,2.67,20.0,2.00,9.0,0.67,10.0,2.00
1391,568323,82.0,96.0,49.0,66.0,72.0,30.0,1.33,8.0,0.67,...,26.0,1.67,7.0,2.33,24.0,1.33,8.0,1.00,10.0,2.33


In [67]:
bdi3_entry_exit

,TEIDS Child ID,bdi_flag,Adaptive Developmental Quotient,Social-Emotional Developmental Quotient,Communication Developmental Quotient,Motor Developmental Quotient,Cognitive Developmental Quotient,Adaptive-Self Care RS,Adaptive-Self Care Z-Score,Adaptive-Personal Responsibility RS,...,Motor-Fine Motor RS,Motor-Fine Motor Z-Score,Motor-Perceptual Motor RS,Motor-Perceptual Motor Z-Score,Cognitive-Attention and Memory RS,Cognitive-Attention and Memory Z-Score,Cognitive-Reasoning / Academic Skills RS,Cognitive-Reasoning / Academic Skills Z-Score,Cognitive-Perception and Concepts RS,Cognitive-Perception and Concepts Z-Score
12,486692,3,85.0,114.0,79.0,91.0,82.0,36.0,1.33,13.0,...,33.0,0.33,15.0,0.33,31.0,0.67,14.0,0.67,13.0,1.67
18,490073,3,94.0,102.0,94.0,96.0,92.0,26.0,1.67,17.0,...,30.0,0.67,12.0,0.33,27.0,0.67,15.0,0.33,13.0,1.00
19,490134,3,61.0,78.0,52.0,87.0,68.0,23.0,2.67,6.0,...,32.0,0.33,12.0,1.00,25.0,1.33,10.0,1.00,10.0,3.00
22,490333,3,79.0,94.0,70.0,94.0,84.0,31.0,1.00,5.0,...,26.0,1.67,19.0,1.67,26.0,1.00,10.0,0.67,14.0,1.00
23,490333,3,97.0,94.0,76.0,98.0,88.0,34.0,0.67,13.0,...,31.0,0.33,16.0,0.67,27.0,0.67,9.0,0.67,15.0,0.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3901,564474,3,46.0,52.0,46.0,42.0,46.0,0.0,3.00,0.0,...,3.0,3.00,0.0,3.00,4.0,3.00,1.0,3.00,5.0,3.00
3902,564596,3,100.0,106.0,97.0,115.0,98.0,35.0,0.67,15.0,...,35.0,1.00,17.0,1.00,31.0,0.33,14.0,0.33,16.0,0.33
3903,565093,3,100.0,88.0,88.0,94.0,98.0,36.0,0.33,13.0,...,30.0,0.67,14.0,0.33,29.0,0.33,15.0,0.33,16.0,0.33
3904,565305,3,115.0,98.0,100.0,89.0,98.0,47.0,1.00,15.0,...,30.0,0.67,12.0,0.33,27.0,0.67,11.0,0.33,22.0,0.67


In [69]:
bdi2_entry_exit

,TEIDS Child ID,bdi_flag,Adaptive Developmental Quotient,Social-Emotional Developmental Quotient,Communication Developmental Quotient,Motor Developmental Quotient,Cognitive Developmental Quotient,Adaptive-Self Care RS,Adaptive-Self Care Z-Score,Adaptive-Personal Responsibility RS,...,Motor-Fine Motor RS,Motor-Fine Motor Z-Score,Motor-Perceptual Motor RS,Motor-Perceptual Motor Z-Score,Cognitive-Attention and Memory RS,Cognitive-Attention and Memory Z-Score,Cognitive-Reasoning / Academic Skills RS,Cognitive-Reasoning / Academic Skills Z-Score,Cognitive-Perception and Concepts RS,Cognitive-Perception and Concepts Z-Score
0,474848,2,64.0,86.0,94.0,98.0,88.0,27.0,2.33,6.0,...,39.0,0.33,19.0,0.33,30.0,0.67,17.0,0.00,17.0,1.33
1,474967,2,73.0,92.0,61.0,66.0,70.0,30.0,2.00,10.0,...,31.0,1.00,10.0,2.00,28.0,1.00,7.0,1.67,12.0,2.33
2,475907,2,73.0,86.0,70.0,74.0,64.0,43.0,0.67,5.0,...,28.0,1.67,12.0,1.33,22.0,2.00,12.0,1.00,11.0,3.00
3,476193,2,46.0,78.0,52.0,42.0,58.0,17.0,3.00,0.0,...,19.0,3.00,2.0,3.00,28.0,1.00,0.0,3.00,8.0,3.00
4,478004,2,85.0,104.0,97.0,100.0,102.0,29.0,2.00,17.0,...,35.0,0.00,17.0,0.33,36.0,0.00,16.0,0.00,28.0,0.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2960,545411,2,76.0,82.0,49.0,74.0,78.0,28.0,1.33,4.0,...,26.0,1.33,10.0,1.33,26.0,0.67,8.0,1.00,10.0,2.00
2963,545428,2,85.0,66.0,64.0,85.0,74.0,31.0,1.00,8.0,...,28.0,1.00,14.0,0.33,22.0,1.67,6.0,1.33,12.0,1.33
2966,545473,2,76.0,68.0,82.0,79.0,90.0,32.0,1.00,4.0,...,28.0,1.00,13.0,0.00,25.0,1.00,14.0,0.33,14.0,1.00
2976,545599,2,105.0,100.0,103.0,103.0,103.0,29.0,0.33,NaN,...,24.0,0.67,NaN,NaN,26.0,0.00,NaN,NaN,12.0,0.33


In [71]:
bdi3_merge = bdi3_entry.merge(bdi3_entry_exit,how='inner',on='TEIDS Child ID',suffixes=("_entry","_exit"))

In [73]:
bdi3_merge

,TEIDS Child ID,Adaptive Developmental Quotient_entry,Social-Emotional Developmental Quotient_entry,Communication Developmental Quotient_entry,Motor Developmental Quotient_entry,Cognitive Developmental Quotient_entry,Adaptive-Self Care RS_entry,Adaptive-Self Care Z-Score_entry,Adaptive-Personal Responsibility RS_entry,Adaptive-Personal Responsibility Z-Score_entry,...,Motor-Fine Motor RS_exit,Motor-Fine Motor Z-Score_exit,Motor-Perceptual Motor RS_exit,Motor-Perceptual Motor Z-Score_exit,Cognitive-Attention and Memory RS_exit,Cognitive-Attention and Memory Z-Score_exit,Cognitive-Reasoning / Academic Skills RS_exit,Cognitive-Reasoning / Academic Skills Z-Score_exit,Cognitive-Perception and Concepts RS_exit,Cognitive-Perception and Concepts Z-Score_exit
0,486692,67.0,106.0,79.0,94.0,92.0,27.0,1.67,2.0,2.00,...,33.0,0.33,15.0,0.33,31.0,0.67,14.0,0.67,13.0,1.67
1,490073,67.0,88.0,64.0,70.0,64.0,19.0,2.00,2.0,1.67,...,30.0,0.67,12.0,0.33,27.0,0.67,15.0,0.33,13.0,1.00
2,490134,70.0,88.0,49.0,87.0,84.0,24.0,1.67,3.0,1.67,...,32.0,0.33,12.0,1.00,25.0,1.33,10.0,1.00,10.0,3.00
3,490333,79.0,94.0,70.0,94.0,84.0,31.0,1.00,5.0,1.33,...,26.0,1.67,19.0,1.67,26.0,1.00,10.0,0.67,14.0,1.00
4,490333,79.0,94.0,70.0,94.0,84.0,31.0,1.00,5.0,1.33,...,31.0,0.33,16.0,0.67,27.0,0.67,9.0,0.67,15.0,0.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,564474,46.0,48.0,46.0,42.0,46.0,0.0,3.00,0.0,3.00,...,3.0,3.00,0.0,3.00,4.0,3.00,1.0,3.00,5.0,3.00
1306,564596,97.0,102.0,76.0,111.0,100.0,29.0,0.67,11.0,0.33,...,35.0,1.00,17.0,1.00,31.0,0.33,14.0,0.33,16.0,0.33
1307,565093,88.0,76.0,67.0,104.0,100.0,29.0,0.67,6.0,0.67,...,30.0,0.67,14.0,0.33,29.0,0.33,15.0,0.33,16.0,0.33
1308,565305,106.0,84.0,61.0,102.0,98.0,35.0,0.00,13.0,0.67,...,30.0,0.67,12.0,0.33,27.0,0.67,11.0,0.33,22.0,0.67


In [75]:
bdi3_oc1_e = bdi3_merge[(bdi3_merge['Social-Emotional Developmental Quotient_entry']>=78)
    &(bdi3_merge['Social Emotional-Adult Interaction Z-Score_entry']>-1.5)
    &(bdi3_merge['Social Emotional-Self Concept / Social Role Z-Score_entry']>-1.5)
    &(bdi3_merge['Social Emotional-Peer Interaction Z-Score_entry']>-1.5)]

In [77]:
bdi3_oc1 = ['TEIDS Child ID','Social-Emotional Developmental Quotient_entry','Social-Emotional Developmental Quotient_exit','Social Emotional-Adult Interaction Z-Score_entry','Social Emotional-Adult Interaction Z-Score_exit','Social Emotional-Self Concept / Social Role Z-Score_entry','Social Emotional-Self Concept / Social Role Z-Score_exit','Social Emotional-Peer Interaction Z-Score_entry','Social Emotional-Peer Interaction Z-Score_exit','Social Emotional-Adult Interaction RS_entry','Social Emotional-Adult Interaction RS_exit','Social Emotional-Peer Interaction RS_entry','Social Emotional-Peer Interaction RS_exit','Social Emotional-Self Concept / Social Role RS_entry','Social Emotional-Self Concept / Social Role RS_exit']

In [79]:
bdi3_oc1_e['oc1']='e'

C:\Users\mrmik\AppData\Local\Temp\ipykernel_178664\2194027183.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bdi3_oc1_e['oc1']='e'


In [81]:
bdi3_oc1_e['oc1']

0       e
1       e
2       e
3       e
4       e
       ..
1299    e
1300    e
1302    e
1306    e
1308    e
Name: oc1, Length: 667, dtype: object

In [83]:
bdi3_oc1_d = bdi3_merge[((bdi3_merge['Social-Emotional Developmental Quotient_entry']<78)&(bdi3_merge['Social-Emotional Developmental Quotient_exit']>=78))
    |(((bdi3_merge['Social-Emotional Developmental Quotient_entry']>=78)&(bdi3_merge['Social-Emotional Developmental Quotient_exit']>=78))
      &((bdi3_merge['Social Emotional-Adult Interaction Z-Score_entry']<=-1.5)
        |(bdi3_merge['Social Emotional-Adult Interaction Z-Score_exit']<=-1.5)
        |(bdi3_merge['Social Emotional-Self Concept / Social Role Z-Score_entry']<=-1.5)
        |(bdi3_merge['Social Emotional-Self Concept / Social Role Z-Score_exit']<=-1.5)
        |(bdi3_merge['Social Emotional-Peer Interaction Z-Score_entry']<=-1.5)
        |(bdi3_merge['Social Emotional-Peer Interaction Z-Score_exit']<=-1.5)))]

In [85]:
bdi3_oc1_d['oc1'] = 'd'

C:\Users\mrmik\AppData\Local\Temp\ipykernel_178664\3226782358.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bdi3_oc1_d['oc1'] = 'd'


In [87]:
bdi3_oc1_c = bdi3_merge[((bdi3_merge['Social-Emotional Developmental Quotient_entry']<bdi3_merge['Social-Emotional Developmental Quotient_exit'])
                         &(bdi3_merge['Social-Emotional Developmental Quotient_exit']<78))
    &((bdi3_merge['Social Emotional-Adult Interaction RS_exit']>bdi3_merge['Social Emotional-Adult Interaction RS_entry'])
      |(bdi3_merge['Social Emotional-Peer Interaction RS_exit']>bdi3_merge['Social Emotional-Peer Interaction RS_entry'])
      |(bdi3_merge['Social Emotional-Self Concept / Social Role RS_exit']>bdi3_merge['Social Emotional-Self Concept / Social Role RS_entry']))]

In [89]:
bdi3_oc1_c['oc1']='c'

C:\Users\mrmik\AppData\Local\Temp\ipykernel_178664\2207773202.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bdi3_oc1_c['oc1']='c'


In [91]:
bdi3_oc1_b = bdi3_merge[((bdi3_merge['Social-Emotional Developmental Quotient_entry']>=bdi3_merge['Social-Emotional Developmental Quotient_exit'])
                         &(bdi3_merge['Social-Emotional Developmental Quotient_exit']<78))
    &((bdi3_merge['Social Emotional-Adult Interaction RS_exit']>bdi3_merge['Social Emotional-Adult Interaction RS_entry'])
      |(bdi3_merge['Social Emotional-Peer Interaction RS_exit']>bdi3_merge['Social Emotional-Peer Interaction RS_entry'])
      |(bdi3_merge['Social Emotional-Self Concept / Social Role RS_exit']>bdi3_merge['Social Emotional-Self Concept / Social Role RS_entry']))]

In [93]:
bdi3_oc1_b['oc1']='b'

C:\Users\mrmik\AppData\Local\Temp\ipykernel_178664\688925823.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bdi3_oc1_b['oc1']='b'


In [95]:
bdi3_oc1_a = bdi3_merge[(bdi3_merge['Social-Emotional Developmental Quotient_exit']<78)
    &(bdi3_merge['Social Emotional-Adult Interaction RS_exit']<=bdi3_merge['Social Emotional-Adult Interaction RS_entry'])
    &(bdi3_merge['Social Emotional-Peer Interaction RS_exit']<=bdi3_merge['Social Emotional-Peer Interaction RS_entry'])
    &(bdi3_merge['Social Emotional-Self Concept / Social Role RS_exit']<=bdi3_merge['Social Emotional-Self Concept / Social Role RS_entry'])]

In [97]:
#bdi3_oc1_a.to_csv('bdi3_oc1_a', index=False)

In [99]:
#bdi_oc1_a_output = pd.concat([bdi3_oc1_a, bdi3_oc1_b, bdi3_oc1_c, bdi3_oc1_c, bdi3_oc1_d, bdi3_oc1_e],join='inner')
#bdi_oc1_a_output

In [101]:
bdi3_oc1_a

,TEIDS Child ID,Adaptive Developmental Quotient_entry,Social-Emotional Developmental Quotient_entry,Communication Developmental Quotient_entry,Motor Developmental Quotient_entry,Cognitive Developmental Quotient_entry,Adaptive-Self Care RS_entry,Adaptive-Self Care Z-Score_entry,Adaptive-Personal Responsibility RS_entry,Adaptive-Personal Responsibility Z-Score_entry,...,Motor-Fine Motor RS_exit,Motor-Fine Motor Z-Score_exit,Motor-Perceptual Motor RS_exit,Motor-Perceptual Motor Z-Score_exit,Cognitive-Attention and Memory RS_exit,Cognitive-Attention and Memory Z-Score_exit,Cognitive-Reasoning / Academic Skills RS_exit,Cognitive-Reasoning / Academic Skills Z-Score_exit,Cognitive-Perception and Concepts RS_exit,Cognitive-Perception and Concepts Z-Score_exit
29,500338,67.0,68.0,46.0,59.0,48.0,17.0,2.33,4.0,1.33,...,22.0,2.33,2.0,3.00,18.0,2.33,0.0,3.00,5.0,3.00
51,505750,61.0,74.0,49.0,94.0,70.0,22.0,1.33,0.0,3.00,...,27.0,0.67,12.0,0.33,22.0,1.33,4.0,2.00,9.0,1.67
79,514559,46.0,66.0,46.0,42.0,48.0,5.0,3.00,0.0,3.00,...,11.0,3.00,2.0,3.00,11.0,3.00,0.0,3.00,8.0,3.00
105,526656,67.0,82.0,46.0,91.0,84.0,28.0,1.33,1.0,2.33,...,16.0,3.00,3.0,3.00,22.0,1.67,2.0,3.00,6.0,3.00
114,528517,73.0,80.0,58.0,91.0,82.0,16.0,2.00,4.0,1.00,...,23.0,2.00,11.0,1.00,16.0,2.67,6.0,1.33,8.0,3.00
142,533008,67.0,78.0,52.0,61.0,76.0,16.0,2.00,2.0,1.67,...,24.0,2.00,9.0,1.67,20.0,2.00,8.0,1.00,8.0,3.00
213,542136,61.0,98.0,58.0,79.0,74.0,14.0,2.67,2.0,1.67,...,24.0,2.00,12.0,0.67,24.0,1.33,7.0,1.33,10.0,2.67
254,544076,91.0,96.0,61.0,76.0,58.0,29.0,0.33,5.0,0.67,...,24.0,2.00,6.0,2.67,24.0,1.33,6.0,1.33,9.0,2.67
260,544204,85.0,96.0,85.0,94.0,96.0,28.0,0.67,4.0,1.00,...,26.0,1.67,15.0,0.33,22.0,1.67,8.0,1.00,14.0,1.00
261,544217,55.0,84.0,46.0,72.0,66.0,18.0,2.00,0.0,3.00,...,25.0,1.67,9.0,1.67,22.0,1.67,6.0,1.33,11.0,1.67


In [103]:
eco_df = eco_df.rename(columns={'CHILD_ID': 'TEIDS Child ID'})
eco_df

,DISTRICT,TEIDS Child ID,Child ID_ Assessement point,ECO_Entry_DATE,Ent SOCIAL_SCALE,Ent KNOWLEDGE_SCALE,Ent APPROPRIATE_ACTION_SCALE,ECO_Exit_DATE,Exit SOCIAL_SCALE,Exit KNOWLEDGE_SCALE,...,BDI-2\nEntry SOCIAL_SCALE,BDI-2\nEntry KNOWLEDGE_SCALE,BDI-2\nEntry APPROPRIATE_ACTION_SCALE,BDI2 Entrance S/E\nStd Dev (Zscore),BDI2 Entrance\nCommunication Std Dev. (Z Score),BDI2 Entrance\nCognition Std Dev. (Z Score),BDI2 Entrance\nK/S\nStd Dev (Zscore),BDI2 Entrance\nAdaptive Std Dev (Zscore),BDI2 Entrance\nMotor Std Dev (Zscore),BDI2 Entrance\nA/B\nStd Dev (Zscore)
0,GN,473991,NaN,2020-01-07,1.0,1.0,1.0,2022-12-27,7.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GN,474186,NaN,2020-02-13,1.0,1.0,1.0,2023-10-26,7.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ET,474575,NaN,2020-01-17,3.0,5.0,4.0,2020-07-29,3.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NW,474623,NaN,2019-01-22,3.0,1.0,2.0,2022-12-16,4.0,4.0,...,3.0,6.0,2.0,-1.8,-1.27,-1.33,-1.300,-2.67,-1.67,-2.17
4,SW,474848,NaN,2021-07-09,3.0,3.0,6.0,2023-08-04,7.0,7.0,...,3.0,1.0,6.0,-2.0,-3.00,-2.47,-2.735,-1.67,-0.67,-1.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,SC,579717,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9995,SC,579729,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,ET,580243,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,FT,580375,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
aid3 = bdi3_oc1_a['TEIDS Child ID']
time_in_program = eco_df[['TEIDS Child ID','Days btw Initial and Exit','Days btw I-IFSP to Exit ECO','<Calc> Entrance Age (months)']]
time_in_program = time_in_program[time_in_program['Days btw Initial and Exit'] > 181]
time_in_program.loc[:, 'Years btw Initial and Exit'] = time_in_program['Days btw Initial and Exit'] / 365.2422
time_in_program.loc[:, 'Years btw I-IFSP to Exit ECO'] = time_in_program['Days btw I-IFSP to Exit ECO'] / 365.2422
time_in_program.loc[:, 'Age upon Exit (years)'] = (
    (time_in_program['<Calc> Entrance Age (months)']/12) + time_in_program['Years btw Initial and Exit']
)
exit_age = time_in_program[['TEIDS Child ID','Age upon Exit (years)']]
exit_age_a = exit_age[exit_age['TEIDS Child ID'].isin(aid3)]
exit_age_sorted = exit_age_a.sort_values(by='Age upon Exit (years)', ascending=False)
exit_age_sorted

,TEIDS Child ID,Age upon Exit (years)
1360,505750,5.990993
3215,526656,5.956755
6617,548484,5.076344
6429,546876,5.045031
6691,549176,4.951596
6706,549286,4.885036
6896,551084,4.789209
6245,545247,4.782883
7086,552972,4.746945
5792,542136,4.595510


In [109]:
bdi3_eco_a = eco_df[eco_df['TEIDS Child ID'].isin(aid3)]

bdi3_eco_a['length_between_assessment'] = bdi3_eco_a['ECO_Exit_DATE'] - bdi3_eco_a['ECO_Entry_DATE']

bdi3_eco_diff = bdi3_eco_a[['TEIDS Child ID','length_between_assessment']]

bdi3_eco_diff = bdi3_eco_diff[bdi3_eco_diff['length_between_assessment'].dt.days>181]

C:\Users\mrmik\AppData\Local\Temp\ipykernel_178664\1348050299.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bdi3_eco_a['length_between_assessment'] = bdi3_eco_a['ECO_Exit_DATE'] - bdi3_eco_a['ECO_Entry_DATE']


In [ ]:
# Second objective: Demographic qualities for children who did not improve functioning (progress category A) 

In [357]:
eco_df

,DISTRICT,TEIDS Child ID,Child ID_ Assessement point,ECO_Entry_DATE,Ent SOCIAL_SCALE,Ent KNOWLEDGE_SCALE,Ent APPROPRIATE_ACTION_SCALE,ECO_Exit_DATE,Exit SOCIAL_SCALE,Exit KNOWLEDGE_SCALE,...,BDI-2\nEntry SOCIAL_SCALE,BDI-2\nEntry KNOWLEDGE_SCALE,BDI-2\nEntry APPROPRIATE_ACTION_SCALE,BDI2 Entrance S/E\nStd Dev (Zscore),BDI2 Entrance\nCommunication Std Dev. (Z Score),BDI2 Entrance\nCognition Std Dev. (Z Score),BDI2 Entrance\nK/S\nStd Dev (Zscore),BDI2 Entrance\nAdaptive Std Dev (Zscore),BDI2 Entrance\nMotor Std Dev (Zscore),BDI2 Entrance\nA/B\nStd Dev (Zscore)
0,GN,473991,NaN,2020-01-07,1.0,1.0,1.0,2022-12-27,7.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GN,474186,NaN,2020-02-13,1.0,1.0,1.0,2023-10-26,7.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ET,474575,NaN,2020-01-17,3.0,5.0,4.0,2020-07-29,3.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NW,474623,NaN,2019-01-22,3.0,1.0,2.0,2022-12-16,4.0,4.0,...,3.0,6.0,2.0,-1.8,-1.27,-1.33,-1.300,-2.67,-1.67,-2.17
4,SW,474848,NaN,2021-07-09,3.0,3.0,6.0,2023-08-04,7.0,7.0,...,3.0,1.0,6.0,-2.0,-3.00,-2.47,-2.735,-1.67,-0.67,-1.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,SC,579717,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9995,SC,579729,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,ET,580243,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,FT,580375,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [255]:
# Count of point-of-entry (POE) office for new business rules with Ryan's code

temp_eco = eco_df[eco_df['TEIDS Child ID'].isin(bdi3_oc1_a['TEIDS Child ID'])]
temp_eco

,DISTRICT,TEIDS Child ID,Child ID_ Assessement point,ECO_Entry_DATE,Ent SOCIAL_SCALE,Ent KNOWLEDGE_SCALE,Ent APPROPRIATE_ACTION_SCALE,ECO_Exit_DATE,Exit SOCIAL_SCALE,Exit KNOWLEDGE_SCALE,...,BDI-2\nEntry SOCIAL_SCALE,BDI-2\nEntry KNOWLEDGE_SCALE,BDI-2\nEntry APPROPRIATE_ACTION_SCALE,BDI2 Entrance S/E\nStd Dev (Zscore),BDI2 Entrance\nCommunication Std Dev. (Z Score),BDI2 Entrance\nCognition Std Dev. (Z Score),BDI2 Entrance\nK/S\nStd Dev (Zscore),BDI2 Entrance\nAdaptive Std Dev (Zscore),BDI2 Entrance\nMotor Std Dev (Zscore),BDI2 Entrance\nA/B\nStd Dev (Zscore)
889,UC,500338,NaN,2023-01-27,2.0,1.0,1.0,2023-06-21,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1360,MD,505750,NaN,2021-07-20,5.0,1.0,1.0,2023-07-05,4.0,1.0,...,5.0,1.0,1.0,-1.33,-2.53,-2.67,-2.600,-2.67,-2.67,-2.67
2138,MD,514559,NaN,2023-06-01,2.0,1.0,1.0,2024-04-22,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3215,SE,526656,NaN,2022-05-24,1.0,3.0,4.0,2024-03-22,2.0,1.0,...,7.0,2.0,7.0,-0.67,-3.00,-1.33,-2.165,0.33,-0.13,0.10
3505,GN,528517,NaN,2023-03-01,5.0,3.0,6.0,2023-11-17,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4325,GN,533008,NaN,2023-06-15,5.0,1.0,1.0,2024-02-21,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5792,MD,542136,NaN,2023-01-16,7.0,2.0,3.0,2024-02-15,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6076,GN,544076,NaN,2023-01-18,7.0,1.0,6.0,2023-10-25,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6091,SC,544204,NaN,2023-04-05,7.0,7.0,7.0,2024-01-04,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6092,SW,544217,NaN,2023-02-15,6.0,1.0,1.0,2023-08-22,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [161]:
poe_count = temp_eco.value_counts('DISTRICT')
poe_count = poe_count.reset_index()
poe_count.columns = ['DISTRICT', 'COUNT']
poe_count

,DISTRICT,COUNT
0,GN,13
1,MD,13
2,SC,10
3,ET,5
4,FT,3
5,SE,3
6,SW,2
7,UC,2


In [151]:
mapping = {
    "FT": ("First TN", "37601"),
    "ET": ("East TN", "37920"),
    "SE": ("Southeast TN", "37406"),
    "UC": ("Upper Cumberland", "38506"),
    "GN": ("Greater Nashville", "37243"),
    "SC": ("South Central", "37130"),
    "NW": ("Northwest", "38238"),
    "SW": ("Southwest", "38301"),
    "MD": ("Memphis Delta", "38134")
}

In [215]:
poe_count['DISTRICT NAME'] = poe_count['DISTRICT'].map(lambda x: mapping.get(x, ('Unknown', '00000'))[0])
poe_count['NEARBY CITY ZIP'] = poe_count['DISTRICT'].map(lambda x: mapping.get(x, ('Unknown', '00000'))[1])
poe_count = poe_count[['DISTRICT', 'DISTRICT NAME', 'NEARBY CITY ZIP', 'COUNT']]
poe_count

C:\Users\mrmik\AppData\Local\Temp\ipykernel_178664\2950739031.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poe_count['DISTRICT NAME'] = poe_count['DISTRICT'].map(lambda x: mapping.get(x, ('Unknown', '00000'))[0])
C:\Users\mrmik\AppData\Local\Temp\ipykernel_178664\2950739031.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poe_count['NEARBY CITY ZIP'] = poe_count['DISTRICT'].map(lambda x: mapping.get(x, ('Unknown', '00000'))[1])


,DISTRICT,DISTRICT NAME,NEARBY CITY ZIP,COUNT
0,GN,Greater Nashville,37243,13
1,MD,Memphis Delta,38134,13
2,SC,South Central,37130,10
3,ET,East TN,37920,5
4,FT,First TN,37601,3
5,SE,Southeast TN,37406,3
6,SW,Southwest,38301,2
7,UC,Upper Cumberland,38506,2


In [171]:
eco_df[eco_df['TEIDS Child ID'].isin(bdi3_oc1_a['TEIDS Child ID']) & (eco_df['DISTRICT'] == 'GN')]

,DISTRICT,TEIDS Child ID,Child ID_ Assessement point,ECO_Entry_DATE,Ent SOCIAL_SCALE,Ent KNOWLEDGE_SCALE,Ent APPROPRIATE_ACTION_SCALE,ECO_Exit_DATE,Exit SOCIAL_SCALE,Exit KNOWLEDGE_SCALE,...,BDI-2\nEntry SOCIAL_SCALE,BDI-2\nEntry KNOWLEDGE_SCALE,BDI-2\nEntry APPROPRIATE_ACTION_SCALE,BDI2 Entrance S/E\nStd Dev (Zscore),BDI2 Entrance\nCommunication Std Dev. (Z Score),BDI2 Entrance\nCognition Std Dev. (Z Score),BDI2 Entrance\nK/S\nStd Dev (Zscore),BDI2 Entrance\nAdaptive Std Dev (Zscore),BDI2 Entrance\nMotor Std Dev (Zscore),BDI2 Entrance\nA/B\nStd Dev (Zscore)
3505,GN,528517,NaN,2023-03-01,5.0,3.0,6.0,2023-11-17,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4325,GN,533008,NaN,2023-06-15,5.0,1.0,1.0,2024-02-21,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6076,GN,544076,NaN,2023-01-18,7.0,1.0,6.0,2023-10-25,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6533,GN,547849,NaN,2023-03-14,7.0,6.0,6.0,2024-01-09,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6706,GN,549286,NaN,2023-04-04,7.0,1.0,6.0,2024-05-09,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6753,GN,549820,NaN,2023-04-21,7.0,4.0,6.0,2024-01-25,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6811,GN,550279,NaN,2023-04-28,7.0,1.0,1.0,2024-01-02,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6993,GN,552059,NaN,2023-05-02,6.0,3.0,7.0,2024-01-24,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7086,GN,552972,NaN,2023-05-22,7.0,1.0,6.0,2024-03-21,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7114,GN,553245,NaN,2023-07-17,7.0,3.0,5.0,2024-03-13,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [173]:
eco_df[eco_df['TEIDS Child ID'].isin(bdi3_oc1_a['TEIDS Child ID']) & (eco_df['DISTRICT'] == 'MD')]

,DISTRICT,TEIDS Child ID,Child ID_ Assessement point,ECO_Entry_DATE,Ent SOCIAL_SCALE,Ent KNOWLEDGE_SCALE,Ent APPROPRIATE_ACTION_SCALE,ECO_Exit_DATE,Exit SOCIAL_SCALE,Exit KNOWLEDGE_SCALE,...,BDI-2\nEntry SOCIAL_SCALE,BDI-2\nEntry KNOWLEDGE_SCALE,BDI-2\nEntry APPROPRIATE_ACTION_SCALE,BDI2 Entrance S/E\nStd Dev (Zscore),BDI2 Entrance\nCommunication Std Dev. (Z Score),BDI2 Entrance\nCognition Std Dev. (Z Score),BDI2 Entrance\nK/S\nStd Dev (Zscore),BDI2 Entrance\nAdaptive Std Dev (Zscore),BDI2 Entrance\nMotor Std Dev (Zscore),BDI2 Entrance\nA/B\nStd Dev (Zscore)
1360,MD,505750,NaN,2021-07-20,5.0,1.0,1.0,2023-07-05,4.0,1.0,...,5.0,1.0,1.0,-1.33,-2.53,-2.67,-2.6,-2.67,-2.67,-2.67
2138,MD,514559,NaN,2023-06-01,2.0,1.0,1.0,2024-04-22,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5792,MD,542136,NaN,2023-01-16,7.0,2.0,3.0,2024-02-15,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6245,MD,545247,NaN,2023-01-27,7.0,3.0,7.0,2024-02-08,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6265,MD,545407,NaN,2023-02-07,4.0,3.0,5.0,2023-12-21,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6450,MD,547077,NaN,2023-02-17,6.0,4.0,7.0,2023-09-18,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6585,MD,548218,NaN,2023-03-08,7.0,1.0,2.0,2023-11-28,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6616,MD,548483,NaN,2023-04-06,7.0,2.0,5.0,2023-12-06,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6809,MD,550241,NaN,2023-04-10,7.0,1.0,6.0,2023-12-11,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6896,MD,551084,NaN,2023-05-16,5.0,3.0,6.0,2024-03-11,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [179]:
pip install pgeocode

Note: you may need to restart the kernel to use updated packages.


In [229]:
import pgeocode
nomi = pgeocode.Nominatim('us')

def get_coordinates(zip_code):
    location = nomi.query_postal_code(zip_code)
    return location.latitude, location.longitude

poe_count[['LAT', 'LON']] = poe_count['NEARBY CITY ZIP'].apply(lambda z: pd.Series(get_coordinates(z)))
poe_count

C:\Users\mrmik\AppData\Local\Temp\ipykernel_178664\2511265461.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poe_count[['LAT', 'LON']] = poe_count['NEARBY CITY ZIP'].apply(lambda z: pd.Series(get_coordinates(z)))


,DISTRICT,DISTRICT NAME,NEARBY CITY ZIP,COUNT,LAT,LON
0,GN,Greater Nashville,37243,13,36.1650,-86.7821
1,MD,Memphis Delta,38134,13,35.1845,-89.8574
2,SC,South Central,37130,10,35.8456,-86.3903
3,ET,East TN,37920,5,35.8929,-83.9387
4,FT,First TN,37601,3,36.3339,-82.3408
5,SE,Southeast TN,37406,3,35.0614,-85.2478
6,SW,Southwest,38301,2,35.6102,-88.8140
7,UC,Upper Cumberland,38506,2,36.1819,-85.4408


In [225]:
import folium
from folium.plugins import HeatMap

tn_map = folium.Map(location=[35.5175, -86.5804], zoom_start=7)

count_data = poe_count[['LAT', 'LON', 'COUNT']].dropna().values.tolist()

HeatMap(count_data).add_to(tn_map)
tn_map

In [355]:
pc_a_eco

,DISTRICT,NEARBY CITY ZIP
64,Unknown,00000
223,Unknown,00000
356,Unknown,00000
889,Unknown,00000
1157,Unknown,00000
...,...,...
7675,Unknown,00000
7710,Unknown,00000
7742,Unknown,00000
7788,Unknown,00000


In [ ]:
pd.merge(old_kids_a, poe_count[['DISTRICT', 'NEARBY CITY ZIP']], on='DISTRICT', how='left')

In [353]:
pc_a_eco['DISTRICT'] = pc_a_eco['DISTRICT'].map(lambda x: mapping.get(x, ('Unknown', '00000'))[0])
pc_a_eco['NEARBY CITY ZIP'] = pc_a_eco['DISTRICT'].map(lambda x: mapping.get(x, ('Unknown', '00000'))[1])
pc_a_eco = pc_a_eco[['DISTRICT', 'NEARBY CITY ZIP', 'COUNT']]
pc_a_eco

C:\Users\mrmik\AppData\Local\Temp\ipykernel_178664\1722562861.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pc_a_eco['DISTRICT'] = pc_a_eco['DISTRICT'].map(lambda x: mapping.get(x, ('Unknown', '00000'))[0])
C:\Users\mrmik\AppData\Local\Temp\ipykernel_178664\1722562861.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pc_a_eco['NEARBY CITY ZIP'] = pc_a_eco['DISTRICT'].map(lambda x: mapping.get(x, ('Unknown', '00000'))[1])


KeyError: "['COUNT'] not in index"

In [279]:
old_a_kids = pd.read_csv(r"C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Projects\teis-mikeriello\data\old_rules_a.csv")
old_a_kids.value_counts('DISTRICT')

DISTRICT
GN    359
MD    342
ET    300
SC    299
UC    203
FT    171
SE    166
NW     91
SW     90
Name: count, dtype: int64

In [285]:
old_a_kids

,DISTRICT,CHILD_ID,Child ID_ Assessement point,ECO_Entry_DATE,Ent SOCIAL_SCALE,Ent KNOWLEDGE_SCALE,Ent APPROPRIATE_ACTION_SCALE,ECO_Exit_DATE,Exit SOCIAL_SCALE,Exit KNOWLEDGE_SCALE,...,BDI-2\nEntry SOCIAL_SCALE,BDI-2\nEntry KNOWLEDGE_SCALE,BDI-2\nEntry APPROPRIATE_ACTION_SCALE,BDI2 Entrance S/E\nStd Dev (Zscore),BDI2 Entrance\nCommunication Std Dev. (Z Score),BDI2 Entrance\nCognition Std Dev. (Z Score),BDI2 Entrance\nK/S\nStd Dev (Zscore),BDI2 Entrance\nAdaptive Std Dev (Zscore),BDI2 Entrance\nMotor Std Dev (Zscore),BDI2 Entrance\nA/B\nStd Dev (Zscore)
0,SE,475907,NaN,2020-02-20,3.0,5.0,1.0,2023-11-15,7.0,2.0,...,7.0,4.0,7.0,-1.00,-1.47,-1.80,-1.635,-0.67,-1.00,-0.835
1,MD,476193,NaN,2020-02-24,2.0,2.0,1.0,2023-02-27,5.0,1.0,...,7.0,5.0,7.0,-0.80,-1.67,-1.13,-1.400,0.00,-0.47,-0.235
2,GN,478108,NaN,2022-10-27,3.0,3.0,7.0,2023-10-03,6.0,2.0,...,3.0,3.0,7.0,-1.80,-2.00,-1.67,-1.835,-1.33,0.27,-0.530
3,UC,478244,NaN,2020-03-10,3.0,5.0,1.0,2023-11-07,7.0,4.0,...,7.0,6.0,7.0,-0.67,-1.07,-1.13,-1.100,-1.33,-0.67,-1.000
4,NW,478507,NaN,2022-04-13,5.0,3.0,5.0,2023-03-09,7.0,1.0,...,7.0,2.0,6.0,-0.53,-3.00,-1.40,-2.200,-1.13,-1.40,-1.265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016,GN,565716,NaN,2023-11-01,2.0,1.0,2.0,2024-08-19,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017,MD,566355,NaN,2023-11-09,7.0,2.0,5.0,2024-08-29,5.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018,ET,566496,NaN,2023-11-12,6.0,3.0,1.0,2024-09-16,5.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019,ET,567510,NaN,2023-11-30,7.0,7.0,4.0,2024-10-02,3.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [291]:
old_poe = pd.merge(old_kids_a, poe_count[['DISTRICT', 'NEARBY CITY ZIP']], on='DISTRICT', how='left')

In [311]:
old_poe

,DISTRICT,count,NEARBY CITY ZIP
0,GN,359,37243
1,MD,342,38134
2,ET,300,37920
3,SC,299,37130
4,UC,203,38506
5,FT,171,37601
6,SE,166,37406
7,NW,91,NaN
8,SW,90,38301


In [309]:
def get_coordinates(zip_code):
    location = nomi.query_postal_code(zip_code)
    return location.latitude, location.longitude

old_poe[['LAT', 'LON']] = old_poe['NEARBY CITY ZIP'].apply(lambda z: pd.Series(get_coordinates(z)))
old_poe

ValueError: DataFrame constructor not properly called!

In [307]:
tn_map = folium.Map(location=[35.5175, -86.5804], zoom_start=7)

count_data2 = old_poe[['LAT', 'LON', 'COUNT']].dropna().values.tolist()

HeatMap(count_data2).add_to(tn_map)
tn_map

KeyError: "None of [Index(['LAT', 'LON', 'COUNT'], dtype='object')] are in the [columns]"

In [269]:
poe_count['DISTRICT NAME'] = poe_count['DISTRICT'].map(lambda x: mapping.get(x, ('Unknown', '00000'))[0])
poe_count['NEARBY CITY ZIP'] = poe_count['DISTRICT'].map(lambda x: mapping.get(x, ('Unknown', '00000'))[1])
poe_count = poe_count[['DISTRICT', 'DISTRICT NAME', 'NEARBY CITY ZIP', 'COUNT']]
poe_count

NameError: name 'poe_count_old' is not defined

In [113]:
#length between exit assessment and program exit
bdi3_a_exit = bdi3_eco_a[['Exit Date \nOr end of FY', 'ECO_Exit_DATE']]
bdi3_a_exit['time_between_assessment_and_exit'] = bdi3_a_exit['Exit Date \nOr end of FY'] - bdi3_a_exit['ECO_Exit_DATE']
bdi3_a_exit.sort_values('time_between_assessment_and_exit', ascending=False)

C:\Users\mrmik\AppData\Local\Temp\ipykernel_178664\2438918784.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bdi3_a_exit['time_between_assessment_and_exit'] = bdi3_a_exit['Exit Date \nOr end of FY'] - bdi3_a_exit['ECO_Exit_DATE']


,Exit Date \nOr end of FY,ECO_Exit_DATE,time_between_assessment_and_exit
1360,2024-06-30,2023-07-05,361 days
3505,2024-06-30,2023-11-17,226 days
6585,2024-06-30,2023-11-28,215 days
6616,2024-06-30,2023-12-06,207 days
6429,2024-06-30,2023-12-06,207 days
6827,2024-06-30,2023-12-22,191 days
7154,2024-06-30,2024-01-11,171 days
6993,2024-06-30,2024-01-24,158 days
6753,2024-06-30,2024-01-25,157 days
6896,2024-08-05,2024-03-11,147 days


In [115]:
eco_df[eco_df['TEIDS Child ID'].isin(bdi3_oc1_a['TEIDS Child ID']) & (eco_df['DISTRICT'] == 'GN')]

,DISTRICT,TEIDS Child ID,Child ID_ Assessement point,ECO_Entry_DATE,Ent SOCIAL_SCALE,Ent KNOWLEDGE_SCALE,Ent APPROPRIATE_ACTION_SCALE,ECO_Exit_DATE,Exit SOCIAL_SCALE,Exit KNOWLEDGE_SCALE,...,BDI-2\nEntry SOCIAL_SCALE,BDI-2\nEntry KNOWLEDGE_SCALE,BDI-2\nEntry APPROPRIATE_ACTION_SCALE,BDI2 Entrance S/E\nStd Dev (Zscore),BDI2 Entrance\nCommunication Std Dev. (Z Score),BDI2 Entrance\nCognition Std Dev. (Z Score),BDI2 Entrance\nK/S\nStd Dev (Zscore),BDI2 Entrance\nAdaptive Std Dev (Zscore),BDI2 Entrance\nMotor Std Dev (Zscore),BDI2 Entrance\nA/B\nStd Dev (Zscore)
3505,GN,528517,NaN,2023-03-01,5.0,3.0,6.0,2023-11-17,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4325,GN,533008,NaN,2023-06-15,5.0,1.0,1.0,2024-02-21,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6076,GN,544076,NaN,2023-01-18,7.0,1.0,6.0,2023-10-25,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6533,GN,547849,NaN,2023-03-14,7.0,6.0,6.0,2024-01-09,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6706,GN,549286,NaN,2023-04-04,7.0,1.0,6.0,2024-05-09,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6753,GN,549820,NaN,2023-04-21,7.0,4.0,6.0,2024-01-25,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6811,GN,550279,NaN,2023-04-28,7.0,1.0,1.0,2024-01-02,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6993,GN,552059,NaN,2023-05-02,6.0,3.0,7.0,2024-01-24,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7086,GN,552972,NaN,2023-05-22,7.0,1.0,6.0,2024-03-21,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7114,GN,553245,NaN,2023-07-17,7.0,3.0,5.0,2024-03-13,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [187]:
bdi3_oc1_a

,TEIDS Child ID,Adaptive Developmental Quotient_entry,Social-Emotional Developmental Quotient_entry,Communication Developmental Quotient_entry,Motor Developmental Quotient_entry,Cognitive Developmental Quotient_entry,Adaptive-Self Care RS_entry,Adaptive-Self Care Z-Score_entry,Adaptive-Personal Responsibility RS_entry,Adaptive-Personal Responsibility Z-Score_entry,...,Motor-Fine Motor RS_exit,Motor-Fine Motor Z-Score_exit,Motor-Perceptual Motor RS_exit,Motor-Perceptual Motor Z-Score_exit,Cognitive-Attention and Memory RS_exit,Cognitive-Attention and Memory Z-Score_exit,Cognitive-Reasoning / Academic Skills RS_exit,Cognitive-Reasoning / Academic Skills Z-Score_exit,Cognitive-Perception and Concepts RS_exit,Cognitive-Perception and Concepts Z-Score_exit
29,500338,67.0,68.0,46.0,59.0,48.0,17.0,2.33,4.0,1.33,...,22.0,2.33,2.0,3.00,18.0,2.33,0.0,3.00,5.0,3.00
51,505750,61.0,74.0,49.0,94.0,70.0,22.0,1.33,0.0,3.00,...,27.0,0.67,12.0,0.33,22.0,1.33,4.0,2.00,9.0,1.67
79,514559,46.0,66.0,46.0,42.0,48.0,5.0,3.00,0.0,3.00,...,11.0,3.00,2.0,3.00,11.0,3.00,0.0,3.00,8.0,3.00
105,526656,67.0,82.0,46.0,91.0,84.0,28.0,1.33,1.0,2.33,...,16.0,3.00,3.0,3.00,22.0,1.67,2.0,3.00,6.0,3.00
114,528517,73.0,80.0,58.0,91.0,82.0,16.0,2.00,4.0,1.00,...,23.0,2.00,11.0,1.00,16.0,2.67,6.0,1.33,8.0,3.00
142,533008,67.0,78.0,52.0,61.0,76.0,16.0,2.00,2.0,1.67,...,24.0,2.00,9.0,1.67,20.0,2.00,8.0,1.00,8.0,3.00
213,542136,61.0,98.0,58.0,79.0,74.0,14.0,2.67,2.0,1.67,...,24.0,2.00,12.0,0.67,24.0,1.33,7.0,1.33,10.0,2.67
254,544076,91.0,96.0,61.0,76.0,58.0,29.0,0.33,5.0,0.67,...,24.0,2.00,6.0,2.67,24.0,1.33,6.0,1.33,9.0,2.67
260,544204,85.0,96.0,85.0,94.0,96.0,28.0,0.67,4.0,1.00,...,26.0,1.67,15.0,0.33,22.0,1.67,8.0,1.00,14.0,1.00
261,544217,55.0,84.0,46.0,72.0,66.0,18.0,2.00,0.0,3.00,...,25.0,1.67,9.0,1.67,22.0,1.67,6.0,1.33,11.0,1.67


In [235]:
time_in_program = eco_df[['TEIDS Child ID','Days btw Initial and Exit','Days btw I-IFSP to Exit ECO','<Calc> Entrance Age (months)']]
time_in_program = time_in_program[time_in_program['Days btw Initial and Exit'] > 181]
time_in_program.loc[:, 'Years btw Initial and Exit'] = time_in_program['Days btw Initial and Exit'] / 365.2422
time_in_program.loc[:, 'Years btw I-IFSP to Exit ECO'] = time_in_program['Days btw I-IFSP to Exit ECO'] / 365.2422
time_in_program.loc[:, 'Age upon Exit (years)'] = (
    (time_in_program['<Calc> Entrance Age (months)']/12) + time_in_program['Years btw Initial and Exit']
)

In [237]:
time_in_program

,TEIDS Child ID,Days btw Initial and Exit,Days btw I-IFSP to Exit ECO,<Calc> Entrance Age (months),Years btw Initial and Exit,Years btw I-IFSP to Exit ECO,Age upon Exit (years)
0,473991,1340,1085,47,3.668798,2.970632,7.585465
1,474186,1388,1381,48,3.800218,3.781053,7.800218
2,474575,1327,194,46,3.633206,0.531154,7.466539
3,474623,1338,1059,46,3.663323,2.899446,7.496656
4,474848,887,756,49,2.428526,2.069859,6.511859
...,...,...,...,...,...,...,...
7939,569165,182,-45308,39,0.498299,-124.049193,3.748299
7940,569195,202,-45309,39,0.553058,-124.051930,3.803058
7941,569207,210,-45302,39,0.574961,-124.032765,3.824961
7942,569218,211,-45300,39,0.577699,-124.027289,3.827699


In [249]:
poe_counts2 = time_in_program[time_in_program['TEIDS Child ID'].isin(bdi3_oc1_a['TEIDS Child ID'])]
poe_counts2 = time_in_program.merge(eco_df[['TEIDS Child ID', 'DISTRICT']], on='TEIDS Child ID', how='left')

In [251]:
poe_counts2

,TEIDS Child ID,Days btw Initial and Exit,Days btw I-IFSP to Exit ECO,<Calc> Entrance Age (months),Years btw Initial and Exit,Years btw I-IFSP to Exit ECO,Age upon Exit (years),DISTRICT
0,473991,1340,1085,47,3.668798,2.970632,7.585465,GN
1,474186,1388,1381,48,3.800218,3.781053,7.800218,GN
2,474575,1327,194,46,3.633206,0.531154,7.466539,ET
3,474623,1338,1059,46,3.663323,2.899446,7.496656,NW
4,474848,887,756,49,2.428526,2.069859,6.511859,SW
...,...,...,...,...,...,...,...,...
7939,569165,182,-45308,39,0.498299,-124.049193,3.748299,MD
7940,569195,202,-45309,39,0.553058,-124.051930,3.803058,FT
7941,569207,210,-45302,39,0.574961,-124.032765,3.824961,ET
7942,569218,211,-45300,39,0.577699,-124.027289,3.827699,MD


In [ ]:
# CHARLIE'S CODE BELOW, RECALCULATING DEMOGRAPHICS DATA

In [313]:
outcome_1 = pd.read_csv('../teis-mikeriello/data/outcome1_a.csv')
outcome_2 = pd.read_csv('../teis-mikeriello/data/outcome2_a.csv')
outcome_3 = pd.read_csv('../teis-mikeriello/data/outcome3_a.csv')

pc_a = pd.concat([outcome_1, outcome_2, outcome_3], ignore_index=True)
id3 = pc_a['Student TEIDS ID']

In [341]:
pc_a_eco = eco_df[eco_df['TEIDS Child ID'].isin(id3)]
pc_a_eco

,DISTRICT,TEIDS Child ID,Child ID_ Assessement point,ECO_Entry_DATE,Ent SOCIAL_SCALE,Ent KNOWLEDGE_SCALE,Ent APPROPRIATE_ACTION_SCALE,ECO_Exit_DATE,Exit SOCIAL_SCALE,Exit KNOWLEDGE_SCALE,...,BDI-2\nEntry SOCIAL_SCALE,BDI-2\nEntry KNOWLEDGE_SCALE,BDI-2\nEntry APPROPRIATE_ACTION_SCALE,BDI2 Entrance S/E\nStd Dev (Zscore),BDI2 Entrance\nCommunication Std Dev. (Z Score),BDI2 Entrance\nCognition Std Dev. (Z Score),BDI2 Entrance\nK/S\nStd Dev (Zscore),BDI2 Entrance\nAdaptive Std Dev (Zscore),BDI2 Entrance\nMotor Std Dev (Zscore),BDI2 Entrance\nA/B\nStd Dev (Zscore)
64,MD,486067,NaN,2022-11-15,4.0,4.0,7.0,2023-05-12,4.0,5.0,...,4.0,4.0,7.0,-1.53,-1.67,-1.73,-1.700,-0.47,-1.53,-1.000
223,SC,490309,NaN,2022-10-04,7.0,1.0,4.0,2023-08-03,2.0,1.0,...,7.0,1.0,4.0,-0.80,-3.00,-2.33,-2.665,-0.80,-2.27,-1.535
356,FT,492996,NaN,2021-06-21,7.0,7.0,7.0,2023-08-17,2.0,1.0,...,7.0,7.0,7.0,0.20,-0.53,0.20,-0.165,-0.67,-0.47,-0.570
889,UC,500338,NaN,2023-01-27,2.0,1.0,1.0,2023-06-21,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1157,MD,503483,NaN,2023-01-25,4.0,3.0,6.0,2024-01-29,2.0,1.0,...,4.0,3.0,6.0,-1.67,-2.73,-1.13,-1.930,-1.67,-0.80,-1.235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7675,SC,560477,NaN,2023-08-18,7.0,1.0,4.0,2024-03-14,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7710,SC,561276,NaN,2023-09-06,4.0,1.0,3.0,2024-03-21,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7742,ET,562179,NaN,2023-09-13,5.0,1.0,1.0,2024-04-10,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7788,SW,563483,NaN,2023-09-27,7.0,3.0,7.0,2024-03-20,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [333]:
new_poe_charlie = pc_a_eco['DISTRICT'].value_counts()

DISTRICT
MD    33
GN    27
SC    22
ET    17
UC    12
FT     9
SE     7
NW     5
SW     5
Name: count, dtype: int64

In [337]:
pc_a_eco

,DISTRICT,TEIDS Child ID,Child ID_ Assessement point,ECO_Entry_DATE,Ent SOCIAL_SCALE,Ent KNOWLEDGE_SCALE,Ent APPROPRIATE_ACTION_SCALE,ECO_Exit_DATE,Exit SOCIAL_SCALE,Exit KNOWLEDGE_SCALE,...,BDI-2\nEntry SOCIAL_SCALE,BDI-2\nEntry KNOWLEDGE_SCALE,BDI-2\nEntry APPROPRIATE_ACTION_SCALE,BDI2 Entrance S/E\nStd Dev (Zscore),BDI2 Entrance\nCommunication Std Dev. (Z Score),BDI2 Entrance\nCognition Std Dev. (Z Score),BDI2 Entrance\nK/S\nStd Dev (Zscore),BDI2 Entrance\nAdaptive Std Dev (Zscore),BDI2 Entrance\nMotor Std Dev (Zscore),BDI2 Entrance\nA/B\nStd Dev (Zscore)
64,MD,486067,NaN,2022-11-15,4.0,4.0,7.0,2023-05-12,4.0,5.0,...,4.0,4.0,7.0,-1.53,-1.67,-1.73,-1.700,-0.47,-1.53,-1.000
223,SC,490309,NaN,2022-10-04,7.0,1.0,4.0,2023-08-03,2.0,1.0,...,7.0,1.0,4.0,-0.80,-3.00,-2.33,-2.665,-0.80,-2.27,-1.535
356,FT,492996,NaN,2021-06-21,7.0,7.0,7.0,2023-08-17,2.0,1.0,...,7.0,7.0,7.0,0.20,-0.53,0.20,-0.165,-0.67,-0.47,-0.570
889,UC,500338,NaN,2023-01-27,2.0,1.0,1.0,2023-06-21,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1157,MD,503483,NaN,2023-01-25,4.0,3.0,6.0,2024-01-29,2.0,1.0,...,4.0,3.0,6.0,-1.67,-2.73,-1.13,-1.930,-1.67,-0.80,-1.235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7675,SC,560477,NaN,2023-08-18,7.0,1.0,4.0,2024-03-14,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7710,SC,561276,NaN,2023-09-06,4.0,1.0,3.0,2024-03-21,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7742,ET,562179,NaN,2023-09-13,5.0,1.0,1.0,2024-04-10,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7788,SW,563483,NaN,2023-09-27,7.0,3.0,7.0,2024-03-20,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
